# Pair RDD - Exercises

## Setup Spark environment

In [2]:
from pathlib import Path

installation_folder = Path("/content/spark-3.5.0-bin-hadoop3")

if not installation_folder.exists():

  # Install Java locally
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null

  # Download & decompress Spark
  !wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz -O spark-3.5.0-bin-hadoop3.tgz
  !tar xf spark-3.5.0-bin-hadoop3.tgz

  # Install finspark
  !pip install -q findspark

  # Setup required environment variables
  import os
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

  print("Spark setup finished!")

else:
  print("Skipping Spark setup")

Spark setup finished!


## Prepare the Spark context

In [3]:
# Import findpsark
import findspark

# Configure the environment
findspark.init()

# Import the Spark components required for the context creation
from pyspark import SparkConf, SparkContext

# Configure and create the context
conf = SparkConf()
conf = conf.setAppName('mds-session')
conf = conf.setMaster('local[*]')
sc = SparkContext.getOrCreate(conf=conf)

## EJERCICIO 1: ¿Cuáles son las 10 palabras más frecuentes de El Quijote?

In [9]:
quijote = sc.textFile('quijote.txt')
frecuencias = quijote.flatMap(lambda linea: linea.split()).map(lambda palabra: (palabra, 1))
palabras_frecuencia = frecuencias.reduceByKey(lambda el1, el2: el1 + el2).sortBy(lambda el: el[1], ascending=False)

## EJERCICIO 2: ¿Cuáles son las 10 palabras más frecuentes tras eliminar las 100 más frecuentes?

In [15]:
top_100 = palabras_frecuencia.map(lambda palabra: palabra[0]).take(100)
top_100_bc = sc.broadcast(top_100)
palabras_frecuencia.filter(lambda palabra: palabra[0] not in top_100_bc.value).take(10)

[('allí', 370),
 ('señora', 363),
 ('sé', 361),
 ('entre', 358),
 ('dar', 351),
 ('han', 348),
 ('tanto', 347),
 ('les', 346),
 ('tengo', 343),
 ('todas', 342)]

## EJERCICIO 3: ¿Qué posición ocupan Quijote y Sancho en el ranking de frecuencias?

In [22]:
palabras_indice = palabras_frecuencia.zipWithIndex()
print(palabras_indice.take(5))
palabras_indice_filtro = palabras_indice.filter(lambda palabra: palabra[0][0] in ('Quijote', 'Sancho'))
palabras_indice_filtro.collect()

[(('que', 19429), 0), (('de', 17986), 1), (('y', 15894), 2), (('la', 10200), 3), (('a', 9549), 4)]


[(('Sancho', 950), 42), (('Quijote', 894), 43)]

## Close the Spark context

In [ ]:
sc.stop()